# Softmax Regression

## PyTorch로 Softmax의 비용 함수 구현하기 (low-level)

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [3]:
z = torch.FloatTensor([1, 2, 3])

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
hypothesis.sum() # sum 이 1 인 것을 볼 수 있다.

tensor(1.)

##### 비용 함수 구현

In [7]:
z = torch.rand(3, 5, requires_grad=True) # 임의의 3 x 5 텐서 생성

In [8]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward0>)


In [27]:
y = torch.randint(5, (3,)).long() # 0~4 사이 임의의 값을 갖는 1 x 5 행렬 생성
print(y)

tensor([0, 3, 2])


In [33]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # scatter_ 가 어떤 함수지?


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])

In [34]:
test = torch.FloatTensor([[1, 2, 3],
                          [4, 5, 6],
                          [7, 8, 9]])

In [41]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()# sum : 제거하고 싶은 dimension을 입력
print(cost)

tensor(1.6311, grad_fn=<MeanBackward0>)


## PyTorch로 Softmax의 비용 함수 구현하기 (high-level)

### F.softmax() + torch.log() = F.log_softmax()

In [42]:
# Low-level
torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward0>)

In [43]:
# High-level
F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward0>)

### F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [44]:
# Lov-level
# 첫 번째 수식
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.6311, grad_fn=<MeanBackward0>)

In [46]:
# 두 번째 수식
(y_one_hot * -torch.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.6311, grad_fn=<MeanBackward0>)

In [47]:
# High-level
# 세 번째 수식
F.nll_loss(F.log_softmax(z, dim=1), y) 
# nll_loss : negative log likelihood loss 
# F.nll_loss를 사용할 때는 one-hot 벡터를 넣을 필요없이 실제값을 인자로 사용

tensor(1.6311, grad_fn=<NllLossBackward0>)

In [49]:
# 네 번째 수식
F.cross_entropy(z, y)

tensor(1.6311, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

In [54]:
torch.manual_seed(1)

In [52]:
x_data = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_data = [2, 2, 2, 1, 1, 1, 0, 0] # one-hot 벡터의 인덱스 값이므로 정수
x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data) # 따라서 LongTensor 사용

In [55]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [56]:
y_one_hot = torch.zeros((8, 3))
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [57]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [58]:
nb_epochs = 1000
for epoch in range (nb_epochs + 1):
    
    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}")

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## Training with F.cross_entropy

In [59]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [60]:
nb_epochs = 1000
for epoch in range (nb_epochs + 1):
    
    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    # 비용 함수
    z = x_train.matmul(W) + b          ###################
    cost = F.cross_entropy(z, y_train) ################### 이 부분만 다름
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}")

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## Softmax Regression을 nn.Module로 구현하기

In [61]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3
model = nn.Linear(4, 3)

In [62]:
optimzier = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    prediction = model(x_train)
    
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimzier.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.076240
Epoch  200/1000 Cost: 0.006534
Epoch  300/1000 Cost: 0.000915
Epoch  400/1000 Cost: 0.000000
Epoch  500/1000 Cost: 0.000003
Epoch  600/1000 Cost: 0.000538
Epoch  700/1000 Cost: 0.000000
Epoch  800/1000 Cost: 0.000000
Epoch  900/1000 Cost: 0.000000
Epoch 1000/1000 Cost: 0.000000


## Softmax Regression을 class로 구현하기

In [63]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)

In [64]:
model = SoftmaxClassifierModel()

In [65]:
optimzier = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    prediction = model(x_train)
    
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimzier.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.173988
Epoch  200/1000 Cost: 0.002343
Epoch  300/1000 Cost: 0.000114
Epoch  400/1000 Cost: 0.000330
Epoch  500/1000 Cost: 0.000012
Epoch  600/1000 Cost: 0.000071
Epoch  700/1000 Cost: 0.000046
Epoch  800/1000 Cost: 0.000000
Epoch  900/1000 Cost: 0.000619
Epoch 1000/1000 Cost: 0.000000
